In [1]:
# --- load libaries ---
from semanticgis.abstract import Pipeline
from semanticgis.compilers import python_executor, mermaid, qgis_recipy
from IPython.display import display, Markdown
import folium
# --- Define Pipeline ---
p = Pipeline(name="Pipeline Demo")

## 1.  Declare input and  Milestone Data -

In [2]:
p.vector.declare_input(
    name="Europa_countries",
    source="https://raw.githubusercontent.com/Esbern/gis_demo_data/main/europa.geojson"
)
p.vector.declare_derived(
    name="Danish territorial sea",
    persistent=True
)


## 2. Declare the processing steps

In [3]:
# Isolate the Kingdom of Denmark
dk_handle = p.vector.filter_by_sql(
    input_name="Europa_countries",
    where_clause="SOVEREIGNT = 'Denmark'"
)

# Buffer to find the 12
p.vector.buffer(
    input_step= dk_handle,
    distance= 0.22, # approximately 12 Nautical miles
    output_name="Danish territorial sea"
)

## 3. Visualize and Validate the Plan --

In [4]:
display(Markdown(mermaid.compile(p)))

```mermaid
graph TD;

    node_1["<b>Europa_countries</b><br/>Load: Europa_countries"]
    node_2["Filter SQL: SOVEREIGNT = 'Denmark'"]
    node_3["<b>Danish territorial sea</b><br/>Buffer (0.22)"]

    %% -- Connections --
    node_1 --> node_2
    node_2 --> node_3
```

## 4. Display a QGIS recipy

In [5]:
display(Markdown(qgis_recipy.compile(p)))

# QGIS Recipe: Pipeline Demo

---

### Step 1: Load: Europa_countries

**Action**: Load a vector layer.

* **Details**: From the main menu, go to `Layer > Add Layer > Add Vector Layer...` and browse to the file at `N/A`.

---

### Step 2: Filter SQL: SOVEREIGNT = 'Denmark'

**Operation**: `vector.filter_by_sql`

---

### Step 3: Buffer (0.22)

**Action**: Create a buffer.

* **Details**: Go to the `Processing Toolbox` and search for the **Buffer** tool.
* **Settings**:
    * *Input layer*: `Filter SQL: SOVEREIGNT = 'Denmark'`
    * *Distance*: `0.22`

## 5 As a proof of concept execute the pipline in Python

In [8]:
final_results = python_executor.compile(p, fetch_results=["Danish territorial sea"])


✅ Pipeline execution started. Order: ['node_1', 'node_2', 'node_3']
  -> Executing Step: 'Load: Europa_countries'...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     ✅ Loaded data into table 'tbl_node_1'.
  -> Executing Step: 'Filter SQL: SOVEREIGNT = 'Denmark''...
     ✅ Filtered 'tbl_node_1' into new table 'tbl_node_2'.
  -> Executing Step: 'Buffer (0.22)'...
     INFO: Found geometry column 'geom' for buffering.
     ✅ Buffered 'tbl_node_2' into new table 'tbl_node_3'.
✅ Pipeline execution finished successfully.
     Fetching final  v1 result 'Danish territorial sea' from table 'tbl_node_3'...


## 6. Display the result as a map

In [9]:
m = folium.Map(location=[70, -15], zoom_start=3)
folium.GeoJson( final_results.get("Danish territorial sea"), name="Danish territorial sea").add_to(m)
display(m)